In this notebook we will test the revision module 

In [4]:
## IMPORTs
from utils.task import Task
import json
import os
from src.pipeline.candidate_generation import candidate_generation
from utils.database_utils.db_info import get_db_schema
from dotenv import load_dotenv
from utils.prompt import load_prompt
import tiktoken
from tqdm import tqdm

load_dotenv()

True

In [5]:
# Function to load JSON data
def load_json_data(filepath):
    with open(filepath, 'r') as file:
        data = json.load(file)
    return data


# Function to create task object
def create_task(example):
    return Task(example)

In [6]:
# load the task data
filepath = "C:/Users\yousf\Bureau\ConvergenceAI\CHESS_Impl\data/test/subsampled_test.json"
data = load_json_data(filepath)
# load the retrieved entities
filepath_entities = "C:/Users\yousf\Bureau\ConvergenceAI\CHESS_Impl\data/test/retrieved_entities.json"
retrieved_entities = load_json_data(filepath_entities)
# load the retrieved context
filepath_context = "C:/Users\yousf\Bureau\ConvergenceAI\CHESS_Impl\data/test/retrieved_context.json"
retrieved_context = load_json_data(filepath_context)
# load the selected schema
filepath_query = "C:/Users\yousf\Bureau\ConvergenceAI\CHESS_Impl\data/test/generated_candidate.json"
generated_candidate = load_json_data(filepath_query)

In [7]:
# test candidate generation module with a random sample
index = 1
example = data[index]
task = create_task(example)
db_path = os.getenv("DB_ROOT_PATH") + f"/{task.db_id}/{task.db_id}.sqlite"
model = ("gpt-4")
ans = candidate_generation(task=task, retrieved_entities=retrieved_entities[index],
                           retrieved_context=retrieved_context[index], selected_schema=get_db_schema(db_path),
                           model=model,
                           num_samples=1)
ans

{'chain_of_thought_reasoning': "To answer the question, I need to find the city location of a specific high school in the county of Merced. The school must have a 'Lunch Provision 2' status, and its grade levels should range from 9 to 12. First, I will identify the relevant tables and columns: 'frpm' for the NSLP Provision Status, 'schools' for the city and educational level, and the 'EILCode' to ensure it's a high school. I will join these tables using the 'CDSCode' which is a common key across them. The conditions to filter the data include checking the 'NSLP Provision Status' for 'Lunch Provision 2', 'Low Grade' and 'High Grade' for 9 and 12 respectively, 'County' for 'Merced', and 'EILCode' for 'HS'. The query will join the necessary tables, apply these filters, and select the 'City' column from the 'schools' table.",
 'SQL': "SELECT T3.City FROM frpm T1 JOIN schools T3 ON T1.CDSCode = T3.CDSCode WHERE T1.`NSLP Provision Status` = 'Lunch Provision 2' AND T1.`Low Grade` = '9' AND T1

In [11]:
from utils.database_utils.execute import execute_query

a=execute_query(db_path,ans['SQL'])
b=execute_query(db_path,task.SQL)
print(a)
print(b)

[('Le Grand',)]
[('Le Grand',)]
